## Start

In [ ]:
from internal.initialize_data import *
from internal.LOLOHA import *
from client.client import *
from server.server import *

import progressbar

EPSILON = 1
RADNOM_SEED = 10
DATASET_NUMBER = 2
B = 0.005
DELTA = 0.001
LIMITED_NUMBER = None
EVOLUTION_DOMAIN_SIZE = 360  # in order to Syn.csv
ALPHA = 0.4
epsiolon1 = ALPHA * EPSILON

## Initialize dataset

In [13]:
df = read_evolution_dataset('dataset/Syn.csv')
dataset, evolution_dataset = read_dataset(f'dataset/Data{DATASET_NUMBER}-coarse.dat', df, limited_number=LIMITED_NUMBER)
domains = attributes_domain(f'dataset/Data{DATASET_NUMBER}-coarse.domain')
tau = len(evolution_dataset[0])
number_of_users = len(dataset)

print('dataset[0] is',dataset[0])
print('evolution_dataset[0][:10] is',evolution_dataset[0][:10])
print('tau is', tau)
print('number of users is', number_of_users)

Reading dataset ...
Append evolution data ...
Reading domains ...
dataset[0] is [4, 0, 0, 0, 12, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0]
evolution_dataset[0][:10] is [102, 56, 56, 56, 56, 145, 145, 145, 145, 145]
tau is 120
number of users is 45222


## Real frequency for each data collection $t \in [\tau]$

In [14]:
dic_real_freq = compute_frequency(evolution_dataset, tau, EVOLUTION_DOMAIN_SIZE)
dic_real_freq[0]

array([0.00300739, 0.00260935, 0.00256512, 0.00249878, 0.00271992,
       0.00216709, 0.00300739, 0.00351599, 0.00260935, 0.0030295 ,
       0.00265358, 0.00382557, 0.0033612 , 0.0030295 , 0.00225554,
       0.00298527, 0.00314006, 0.00238822, 0.00287471, 0.00300739,
       0.00274203, 0.00307373, 0.0021892 , 0.0018575 , 0.00391402,
       0.00307373, 0.00170271, 0.00325063, 0.00327274, 0.0023661 ,
       0.00203441, 0.00201229, 0.00128256, 0.00223343, 0.00300739,
       0.00205652, 0.00221131, 0.00276414, 0.00453319, 0.00225554,
       0.00260935, 0.0035381 , 0.00238822, 0.00296316, 0.00245456,
       0.00316218, 0.00364867, 0.00280837, 0.0035381 , 0.00203441,
       0.00283048, 0.00234399, 0.0030295 , 0.00285259, 0.00254301,
       0.00263146, 0.00234399, 0.00329486, 0.00344965, 0.0018575 ,
       0.0026978 , 0.00305161, 0.0033612 , 0.00196807, 0.0026978 ,
       0.00254301, 0.00207863, 0.00280837, 0.00338331, 0.00265358,
       0.00199018, 0.00294105, 0.00331697, 0.00245456, 0.00320

## Reduce domain size by hashing

In [15]:
g = compute_optimal_domain_size(EPSILON, ALPHA)
hashed_evolution_dataset, user_hash_functions = reduce_domain_dataset(evolution_dataset, g)

# Revise the domains (append evolution domain)
domains.append(list(range(g)))

# Client
client_obj = Client(EPSILON, RADNOM_SEED, B, DELTA)
# Server
server_obj = Server(domains)

print_table(evolution_dataset[0][:10], hashed_evolution_dataset[0][:10], 'evolution_dataset', 'hashed_evolution_dataset')

print('user hash functions are', user_hash_functions)

Reducing domain ...
|   evolution_dataset |   hashed_evolution_dataset |
|---------------------|----------------------------|
|                 102 |                          0 |
|                  56 |                          1 |
|                  56 |                          1 |
|                  56 |                          1 |
|                  56 |                          1 |
|                 145 |                          1 |
|                 145 |                          1 |
|                 145 |                          1 |
|                 145 |                          1 |
|                 145 |                          1 |
user hash functions are [1925931563521076910, 2982670553707359355, 2699532836127066525, 1135324017976503702, 6943639805128060125, 2816199504396413461, 5331706169338243537, 6458995032564729685, 7876063411171636008, 7591730244108215414, 4696493119461998834, 6305253677179671077, 5044898027722125132, 1724200221174470611, 5137138192377405839, 8587

## Perturbation with GRR

In [16]:
perturbed_evolution_dataset = perturbation_GRR(hashed_evolution_dataset, g, EPSILON, 0.2)

print_table(hashed_evolution_dataset[0][:10], perturbed_evolution_dataset[0][:10], 'hashed_evolution_dataset', 'perturbed_evolution_dataset')

Perturbation with GRR ...
|   hashed_evolution_dataset |   perturbed_evolution_dataset |
|----------------------------|-------------------------------|
|                          0 |                             1 |
|                          1 |                             0 |
|                          1 |                             1 |
|                          1 |                             1 |
|                          1 |                             1 |
|                          1 |                             0 |
|                          1 |                             1 |
|                          1 |                             1 |
|                          1 |                             0 |
|                          1 |                             0 |


## Normalize Dataset

In [17]:
# normalize to [-1,1]
normalized_dataset = normalize_dataset(dataset, domains)
normalized_evolution_dataset = normalize_dataset(perturbed_evolution_dataset, [list(range(g)) for _ in range(tau)])

print_table(perturbed_evolution_dataset[0][:10], normalized_evolution_dataset[0][:10], 'perturbed_evolution_dataset', 'normalized_evolution_dataset')

Normalizing dataset to [-1,1]
Normalizing dataset to [-1,1]
|   perturbed_evolution_dataset |   normalized_evolution_dataset |
|-------------------------------|--------------------------------|
|                             1 |                              1 |
|                             0 |                             -1 |
|                             1 |                              1 |
|                             1 |                              1 |
|                             1 |                              1 |
|                             0 |                             -1 |
|                             1 |                              1 |
|                             1 |                              1 |
|                             0 |                             -1 |
|                             0 |                             -1 |


## Wheel of Differential

In [18]:
retrieval_dataset = []
retrieval_evolutional_dataset = []

for data, data_e in zip(normalized_dataset, normalized_evolution_dataset):
    perturbed_data = client_obj.send_perturbed_avg_eigenvector(data, data_e)
    retrieval_data = server_obj.received_avg_eigenvector(perturbed_data)
    retrieval_dataset.append(retrieval_data[0][:-1])
    retrieval_evolutional_dataset.append(get_coloumn_dataset(retrieval_data, -1))


## Evaluation

In [19]:
print_table([*normalized_dataset[0], normalized_evolution_dataset[0][0]], [*retrieval_dataset[0], retrieval_evolutional_dataset[0][0]],
            'normalized data', 'retrival data')

print('domain size of retrieval data is',len(retrieval_dataset[0]))
print('tau is', len(retrieval_evolutional_dataset[0]))

|   normalized data |   retrival data |
|-------------------|-----------------|
|         -0.466667 |       -0.466276 |
|         -1        |       -0.99961  |
|         -1        |       -0.99961  |
|         -1        |       -0.99961  |
|          0.6      |        0.60039  |
|         -1        |       -0.99961  |
|         -1        |       -0.99961  |
|         -1        |       -0.99961  |
|         -1        |       -0.99961  |
|         -1        |       -0.99961  |
|         -1        |       -0.99961  |
|         -1        |       -0.99961  |
|         -0.2      |       -0.19961  |
|         -1        |       -0.99961  |
|         -1        |       -0.99961  |
|          1        |        1.00039  |
domain size of retrieval data is 15
tau is 120


In [20]:
# denormalizing
denormalized = denormalize_dataset(retrieval_dataset, domains)
denormalized_evolution_dataset = denormalize_dataset(retrieval_evolutional_dataset, [list(range(g)) for _ in range(tau)])
rounded_evolution_dataset = round_dataset(denormalized_evolution_dataset)


Denormaizing dataset ...
Denormaizing dataset ...
Rounding dataset ...


In [21]:
print_table(dataset[0], denormalized[0], 'original', 'retrieved')
print_table(hashed_evolution_dataset[0][:10], rounded_evolution_dataset[0][:10], 'original evolution', 'retrieved evolution')

print('MSE is', findMSE(normalized_dataset, retrieval_dataset))
_, avg = average_variation_distance(dataset, denormalized)
print('Average Variation Distance is', avg)

|   original |    retrieved |
|------------|--------------|
|          4 |  4.00293     |
|          0 |  0.00117103  |
|          0 |  0.00292757  |
|          0 |  0.00292757  |
|         12 | 12.0029      |
|          0 |  0.00117103  |
|          0 |  0.00253723  |
|          0 |  0.000975856 |
|          0 |  0.000780685 |
|          0 |  0.000195171 |
|          0 |  0.00292757  |
|          0 |  0.00292757  |
|          6 |  6.00293     |
|          0 |  0.00780685  |
|          0 |  0.000195171 |
|   original evolution |   retrieved evolution |
|----------------------|-----------------------|
|                    0 |                     1 |
|                    1 |                     0 |
|                    1 |                     1 |
|                    1 |                     1 |
|                    1 |                     1 |
|                    1 |                     0 |
|                    1 |                     1 |
|                    1 |                     1 |


## Evaluate Frequency Estimation

In [37]:
prog = progressbar.ProgressBar(maxval=tau)
prog.start()

dic_estimate_freq = []
for t in range(tau):
    dic_estimate_freq.append(LOLOHA_Aggregator(get_coloumn_dataset(rounded_evolution_dataset, t), user_hash_functions, EVOLUTION_DOMAIN_SIZE, EPSILON, epsiolon1, ALPHA))
    prog.update(t) 

prog.finish()

print_table(dic_real_freq[0][:10], dic_estimate_freq[0][:10], 'real frequency', 'estimate frequency')

print('MSE of frequency is', findMSE(dic_real_freq, dic_estimate_freq))

KeyboardInterrupt: 